<a href="https://colab.research.google.com/github/aleshark87/data-intensive-project/blob/main/Progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo le librerie necessarie

In [ ]:
import numpy as np
import pandas as pd
import os.path

**Caricamento dei dati**


In [ ]:
file_zip_url = "https://www.kaggle.com/kwadwoofosu/predict-test-scores-of-students/download/"
file_zip_name="scores_data.zip"

if not os.path.exists(file_zip_name):
  from urllib.request import urlretrieve
  urlretrieve(file_zip_url, file_zip_name)
  
  